# RAG: Query Tranformations

Query transformations are a set of approaches focused on re-writing and/or modifying questions for retrieval.

## Why Query Transformations Are Needed

Raw user queries often don't match how information is stored in documents, leading to poor retrieval results.

### Core Problems with Naive RAG

*  **Ambiguous queries**: User questions can be vague or poorly worded for retrieval
* **Semantic mismatch**: Query language differs from document language
* **Limited perspective**: Single query may miss relevant documents
* **Document chunks may contain irrelevant content** that degrades retrieval

## How Query Transformation Solves these problems

### Part 1: Multi-Query Translation

* **Generates 3-5 query variations** using different phrasings and synonyms
* **Retrieves documents for each variation** independently
* **Combines unique results** from all queries
* **Improves retrieval coverage** and increases likelihood of finding relevant documents

### Part 2: RAG Fusion

* **Uses Reciprocal Rank Fusion (RRF)** to rank and combine results from multiple query variations
* **Provides better document ranking** than simple concatenation
* **Handles duplicate results** across queries with improved scoring

### Part 3: Decomposition

* **Breaks complex questions into smaller sub-questions**
* **Enables focused retrieval** for each sub-component
* **Handles multi-faceted questions** more effectively
* **Uses sequential or independent approaches** to answer sub-questions then combine

### Part 4: Step-Back Prompting

* **Creates broader, more general queries** alongside specific ones
* **Captures high-level concepts** when specific queries are too narrow
* **Combines both specific and general context** for comprehensive answers
* **Provides broader understanding** for better reasoning

### Part 5: HyDE (Hypothetical Document Embeddings)

* **Generates hypothetical documents** instead of embedding queries directly
* **Solves query-document semantic mismatch** in vector space
* **Improves document-to-document similarity** matching
* **Better alignment** between query representations and actual documents

## Overall Impact
* **Without Query Transformations**: Limited, potentially irrelevant results. 
* **With Query Transformations**: More comprehensive retrieval, better semantic matching, higher quality answers


In [1]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

In [2]:
import os
from dotenv import load_dotenv
 
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

## Part 1 - Multi Query Translation

<img src="multi-query-translation.png" width="800" height="300"/>

In [11]:
# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings(model="text-embedding-3-small"))

retriever = vectorstore.as_retriever()

In [12]:
from langchain.prompts import ChatPromptTemplate

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_perspectives 
    | ChatOpenAI(model_name="gpt-4o-mini",temperature=0) 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [13]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "What is task decomposition for LLM agents?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union # removes duplicate documents
docs = retrieval_chain.invoke({"question":question})
len(docs)

/tmp/ipykernel_58767/911277764.py:10: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


4

In [14]:
docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refi

In [15]:
from operator import itemgetter
from langchain_openai import ChatOpenAI

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Task decomposition for LLM (large language model) agents involves breaking down complex tasks into smaller, manageable subgoals. This process enables the agent to handle intricate tasks more efficiently. There are several methods for task decomposition:\n\n1. **Chain of Thought (CoT)**: This technique encourages the model to "think step by step," allowing it to utilize more computational resources at test time to simplify hard tasks into smaller, more manageable steps.\n\n2. **Tree of Thoughts**: This method extends CoT by exploring multiple reasoning possibilities at each step. It decomposes the problem into various thought steps and generates multiple thoughts for each step, creating a tree structure. The search process can be conducted using breadth-first search (BFS) or depth-first search (DFS), with each state evaluated by a classifier or through majority voting.\n\n3. **Prompting and Instructions**: Task decomposition can also be achieved through simple prompting (e.g., asking t

## Part 2 - RAG Fusion

<img src="rag-fusion.png" width="800" height="300"/>

In [16]:
from langchain.prompts import ChatPromptTemplate

# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [17]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_rag_fusion 
    | ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [18]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)

3

In [19]:
docs

[(Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and ref

In [20]:
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Task decomposition for LLM (large language model) agents involves breaking down complex tasks into smaller, manageable subgoals. This process enables the agent to handle intricate tasks more efficiently. Techniques such as Chain of Thought (CoT) and Tree of Thoughts are commonly used for task decomposition. \n\n- **Chain of Thought (CoT)** encourages the model to "think step by step," allowing it to decompose difficult tasks into simpler steps, which also provides insight into the model\'s reasoning process.\n- **Tree of Thoughts** extends this concept by exploring multiple reasoning possibilities at each step, creating a tree structure where the problem is decomposed into various thought steps, and multiple thoughts are generated for each step.\n\nTask decomposition can be achieved through simple prompting, task-specific instructions, or human inputs, facilitating a structured approach to problem-solving for LLM agents.'

In [22]:
# Instead of running the full chain, run each step separately
question = "What is task decomposition for LLM agents?"

# Step 1: Generate multiple queries
generate_queries_chain = (
    ChatPromptTemplate.from_template(
        'You are a helpful assistant that generates multiple search queries based on a single input query. \n\nGenerate multiple search queries related to: {question} \n\nOutput (4 queries):'
    ) 
    | ChatOpenAI(model_name="gpt-4o-mini", temperature=0) 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

queries = generate_queries_chain.invoke({"question": question})
print("Generated queries:")
for i, q in enumerate(queries):
    print(f"  {i+1}. {q}")

# Step 2: Retrieve documents for each query
print("\nRetrieving documents for each query:")
all_results = []
for i, query in enumerate(queries):
    docs = retriever.invoke(query)
    print(f"\nQuery {i+1}: '{query}'")
    print(f"  Retrieved {len(docs)} documents")
    for j, doc in enumerate(docs[:2]):  # Show first 2 docs
        print(f"    Doc {j+1}: {doc.page_content[:100]}...")
    all_results.append(docs)

# Step 3: Apply RRF and get unique union
fused_results = reciprocal_rank_fusion(all_results)
print(f"\nAfter RRF: {len(fused_results)} unique documents")
for i, (doc, score) in enumerate(fused_results[:3]):
    print(f"  Rank {i+1} (score: {score:.4f}): {doc.page_content[:100]}...")

# Step 4: Final answer generation
context = "\n\n".join([doc.page_content for doc, _ in fused_results[:5]])
final_prompt = f"Answer the following question based on this context:\n\n{context}\n\nQuestion: {question}"
print(f"\nContext length: {len(context)} characters")

Generated queries:
  1. 1. How does task decomposition work in large language model (LLM) agents?
  2. 2. Benefits of task decomposition for LLM agents in AI applications.
  3. 3. Examples of task decomposition techniques used in LLM agents.
  4. 4. Comparing task decomposition methods for improving LLM agent performance.

Retrieving documents for each query:

Query 1: '1. How does task decomposition work in large language model (LLM) agents?'
  Retrieved 4 documents
    Doc 1: LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author...
    Doc 2: LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author...

Query 2: '2. Benefits of task decomposition for LLM agents in AI applications.'
  Retrieved 4 documents
    Doc 1: LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author...
    Doc 2: LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimate

## Part 3 - Decomposition

In [23]:
from langchain.prompts import ChatPromptTemplate

# Decomposition
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [24]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Chain
generate_queries_decomposition = ( prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))

# Run
question = "What are the main components of an LLM-powered autonomous agent system?"
questions = generate_queries_decomposition.invoke({"question":question})

In [25]:
questions

['1. What are the key components of a large language model (LLM) used in autonomous agent systems?',
 '2. How do perception and decision-making modules integrate with LLMs in autonomous agents?',
 '3. What role does reinforcement learning play in enhancing LLM-powered autonomous agent systems?']

**Answer Recursively**

<img src="answers-recursively.png" width="800" height="300"/>

In [26]:
# Prompt
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [27]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

def format_qa_pair(question, answer):
    """Format Q and A pair"""
    
    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()

# llm
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

q_a_pairs = ""
for q in questions:
    
    rag_chain = (
    {"context": itemgetter("question") | retriever, 
     "question": itemgetter("question"),
     "q_a_pairs": itemgetter("q_a_pairs")} 
    | decomposition_prompt
    | llm
    | StrOutputParser())

    answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

In [28]:
q_a_pairs

"\n---\nQuestion: 1. What are the key components of a large language model (LLM) used in autonomous agent systems?\nAnswer: The key components of a large language model (LLM) used in autonomous agent systems include:\n\n1. **Planning**: This involves the ability of the agent to break down large tasks into smaller, manageable subgoals, which facilitates efficient handling of complex tasks.\n\n2. **Subgoal and Decomposition**: The agent can decompose tasks into subgoals, allowing for a structured approach to problem-solving.\n\n3. **Reflection and Refinement**: The agent is capable of self-criticism and self-reflection over past actions, enabling it to learn from mistakes and refine its approach for future tasks, thereby improving the quality of its results.\n\n4. **Memory**: The system may incorporate memory components to retain information over time, although challenges exist due to the finite context length of LLMs, which limits the inclusion of historical information and detailed ins

In [29]:
answer

"Reinforcement learning (RL) plays a significant role in enhancing LLM-powered autonomous agent systems by providing a framework for these agents to learn from interactions with their environment and improve their decision-making capabilities over time. Here are several key aspects of how RL contributes to these systems:\n\n1. **Learning from Feedback**: In RL, agents learn by receiving feedback in the form of rewards or penalties based on their actions. This feedback loop allows LLM-powered agents to evaluate the effectiveness of their decisions and refine their strategies accordingly. By incorporating RL, agents can optimize their performance on tasks by understanding which actions lead to better outcomes.\n\n2. **Exploration and Exploitation**: RL encourages a balance between exploration (trying new actions to discover their effects) and exploitation (choosing the best-known actions based on past experiences). This dynamic is crucial for LLM-powered agents, as it enables them to ada

**Answer Individually**

<img src="answer_individually.png" width="800" height="300">

In [30]:
# Answer each sub-question individually 

from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# RAG prompt
prompt_rag = hub.pull("rlm/rag-prompt")

def retrieve_and_rag(question,prompt_rag,sub_question_generator_chain):
    """RAG on each sub-question"""
    
    # Use our decomposition / 
    sub_questions = sub_question_generator_chain.invoke({"question":question})
    
    # Initialize a list to hold RAG chain results
    rag_results = []
    
    for sub_question in sub_questions:
        
        # Retrieve documents for each sub-question
        retrieved_docs = retriever.get_relevant_documents(sub_question)
        
        # Use retrieved documents and sub-question in RAG chain
        answer = (prompt_rag | llm | StrOutputParser()).invoke({"context": retrieved_docs, 
                                                                "question": sub_question})
        rag_results.append(answer)
    
    return rag_results,sub_questions

# Wrap the retrieval and RAG process in a RunnableLambda for integration into a chain
answers, questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)

/tmp/ipykernel_58767/3122629327.py:24: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(sub_question)


In [31]:
print("DECOMPOSED QUESTIONS AND ANSWERS")

for i, (q, a) in enumerate(zip(questions, answers), 1):
    print(f"\nQ{i}: {q}")
    print(f"A{i}: {a}")

DECOMPOSED QUESTIONS AND ANSWERS

Q1: 1. What are the key components of a large language model (LLM) used in autonomous agent systems?
A1: The key components of a large language model (LLM) used in autonomous agent systems include planning, subgoal decomposition, and reflection and refinement. These components enable the agent to break down complex tasks into manageable parts, learn from past actions, and improve future performance. Additionally, memory plays a crucial role in enhancing the agent's capabilities.

Q2: 2. How do perception and decision-making modules integrate with LLMs in autonomous agents?
A2: Perception and decision-making modules in LLM-powered autonomous agents integrate by allowing the LLM to function as the agent's brain, facilitating planning and self-reflection. The agent can break down complex tasks into manageable subgoals and learn from past actions to improve future decisions. Additionally, approaches like ReAct combine reasoning and acting, enabling the LLM

In [32]:
def format_qa_pairs(questions, answers):
    """Format Q and A pairs"""
    
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

context = format_qa_pairs(questions, answers)

# Prompt
template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":context,"question":question})

"An LLM-powered autonomous agent system comprises several main components that work together to enhance its functionality and performance. Key components include:\n\n1. **Large Language Model (LLM)**: The LLM serves as the core processing unit, functioning as the agent's brain. It facilitates planning, self-reflection, and the decomposition of complex tasks into manageable subgoals.\n\n2. **Perception Module**: This module allows the agent to perceive and interpret its environment, providing the necessary input for decision-making processes.\n\n3. **Decision-Making Module**: Integrated with the LLM, this module enables the agent to make informed decisions based on its perceptions and learned experiences.\n\n4. **Reinforcement Learning**: This component enhances the agent's ability to learn from its actions through self-criticism and reflection, allowing it to refine strategies and improve performance over time.\n\n5. **Memory**: Memory plays a crucial role in storing past experiences a

## Part 4 - Step Back 

In [33]:
# Few Shot Examples
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?", #more abstract
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

In [35]:
generate_queries_step_back = prompt | ChatOpenAI(model_name="gpt-4o-mini", temperature=0) | StrOutputParser()
question = "What is task decomposition for LLM agents?"
generate_queries_step_back.invoke({"question": question})

'what is the process of breaking down tasks for agents?'

In [37]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# Response prompt 
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": generate_queries_step_back | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
    | StrOutputParser()
)

chain.invoke({"question": question})

'Task decomposition for LLM (large language model) agents refers to the process of breaking down complex tasks into smaller, more manageable subgoals or steps. This approach is essential for enabling LLM agents to handle intricate tasks efficiently and effectively. \n\nThere are several techniques and methodologies for task decomposition:\n\n1. **Chain of Thought (CoT)**: This technique encourages the model to "think step by step," allowing it to utilize more computational resources during test time. By decomposing a complex task into simpler, smaller tasks, CoT not only enhances the model\'s performance but also provides insights into its reasoning process.\n\n2. **Tree of Thoughts**: An extension of CoT, this method explores multiple reasoning possibilities at each step. It decomposes the problem into various thought steps and generates multiple thoughts for each step, forming a tree structure. The search process can be conducted using either breadth-first search (BFS) or depth-first

In [39]:
question = "What is task decomposition for LLM agents?"
input_data = {"question": question}

print(f"ORIGINAL QUESTION: {question}")
print("="*80)

# Step 1: Generate step-back query
print("\nGENERATING STEP-BACK QUERY")
step_back_query = generate_queries_step_back.invoke(input_data)
print(f"Step-back query: {step_back_query}")

# Step 2: Retrieve context using normal question
print("\nRETRIEVING WITH NORMAL QUESTION")
normal_docs = retriever.invoke(question)
normal_context = "\n\n".join([doc.page_content for doc in normal_docs])
print(f"Normal retrieval: {len(normal_docs)} documents")
print(f"Normal context preview: {normal_context[:200]}...")

# Step 3: Retrieve context using step-back question
print("\nRETRIEVING WITH STEP-BACK QUESTION")
step_back_docs = retriever.invoke(step_back_query)
step_back_context = "\n\n".join([doc.page_content for doc in step_back_docs])
print(f"Step-back retrieval: {len(step_back_docs)} documents")
print(f"Step-back context preview: {step_back_context[:200]}...")

# Step 4: Build final prompt
print("\nBUILDING RESPONSE PROMPT")
prompt_inputs = {
    "normal_context": normal_context,
    "step_back_context": step_back_context,
    "question": question
}
formatted_prompt = response_prompt.invoke(prompt_inputs)
print(f"Final prompt length: {len(formatted_prompt.messages[0].content)} characters")
print(f"Final prompt preview: {formatted_prompt.messages[0].content[:400]}...")

# Step 5: Get final answer
print("\nGENERATING FINAL ANSWER")
final_answer = ChatOpenAI(model_name="gpt-4o-mini", temperature=0).invoke(formatted_prompt)
print(f"Final answer: {final_answer.content}")

ORIGINAL QUESTION: What is task decomposition for LLM agents?

GENERATING STEP-BACK QUERY
Step-back query: what does task decomposition mean in the context of agents?

RETRIEVING WITH NORMAL QUESTION
Normal retrieval: 4 documents
Normal context preview: Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a s...

RETRIEVING WITH STEP-BACK QUESTION
Step-back retrieval: 4 documents
Step-back context preview: Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a s...

BUILDING RESPONSE PROMPT
Final prompt length: 9786 characters
Final prompt preview: You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context

## Part 5 - HyDE (Hypothetical Document Embedding)

<img src="HyDE.png" width="800" height="600"/>

In [40]:
from langchain.prompts import ChatPromptTemplate

# HyDE document genration
template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_docs_for_retrieval = (
    prompt_hyde | ChatOpenAI(model_name="gpt-4o-mini", temperature=0) | StrOutputParser() 
)

# Run
question = "What is task decomposition for LLM agents?"
generate_docs_for_retrieval.invoke({"question":question})

'**Task Decomposition for LLM Agents**\n\nTask decomposition is a critical strategy employed in the development and application of Large Language Model (LLM) agents, facilitating the breakdown of complex tasks into manageable sub-tasks. This approach enhances the efficiency and effectiveness of LLMs in various applications, including natural language understanding, dialogue systems, and automated reasoning.\n\nAt its core, task decomposition involves identifying the constituent components of a larger task and systematically addressing each component. For LLM agents, this process can be categorized into several stages: task analysis, sub-task identification, and execution strategy formulation. During task analysis, the primary objective is to understand the overarching goal and the specific requirements necessary for its completion. This may involve parsing the task into distinct phases, such as information retrieval, data processing, and response generation.\n\nOnce the task is analyze

In [41]:
# Retrieve
retrieval_chain = generate_docs_for_retrieval | retriever 
retireved_docs = retrieval_chain.invoke({"question":question})
retireved_docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Component One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a

In [42]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":retireved_docs,"question":question})

'Task decomposition for LLM (large language model) agents involves breaking down complex tasks into smaller, manageable subgoals. This process enables the agent to handle complicated tasks more efficiently. There are several methods for task decomposition:\n\n1. **Chain of Thought (CoT)**: This technique encourages the model to "think step by step," allowing it to utilize more computational resources at test time to decompose difficult tasks into simpler steps. CoT transforms large tasks into multiple manageable tasks and provides insight into the model\'s reasoning process.\n\n2. **Tree of Thoughts**: This method extends CoT by exploring multiple reasoning possibilities at each step. It decomposes the problem into various thought steps and generates multiple thoughts for each step, creating a tree structure. The search process can be conducted using breadth-first search (BFS) or depth-first search (DFS), with each state evaluated by a classifier or through majority voting.\n\n3. **Pro